In [1]:
import sys
from alphaPhosHelperFunctions import *
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import plotly.figure_factory as ff
from PeptideCollapse import *
import analytics_core_V04 as ac
import kinase_library as kl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from scipy import stats
import gseapy as gp
from gseapy import dotplot, barplot, enrichment_map
import math
sys.path.insert(1, 'D:\Projects\nanoPhos_env\src\alphaquant')
import alphaquant.run_pipeline as aq_pipeline

# Custom functions

In [2]:
class PhosphoAnalysis:
    def __init__(self):
        self.processor = PeptideCollapse()
        self.condition_df = None
        self.collapsed_data = None
        self.formatted_data = None
        
    
    def peptideCollapse(self, data, **kwargs):
        
        self.collapsed_data = self.processor.process_complete_pipeline(
            data=data, 
            **kwargs
        )
        
        return self.collapsed_data 
        
    def assign_condition_setup(self, condition_df=None):
        
        import warnings
        warnings.filterwarnings('ignore', category=FutureWarning)
        
        df_copy = self.collapsed_data.copy()
        
        if condition_df is None:
            condition_df = self.processor.get_precursor_condition_dataset() 
            
            
            print("Assign conditions to each sample")
        
            for i, sample in enumerate(condition_df['Sample']):
                condition = input(f"Enter condition for '{sample}': ")
                condition_df.loc[i, 'Condition'] = condition
            
            quant_cols = condition_df['Sample'].unique().tolist()
            meta_cols = [col for col in df_copy.columns if col not in quant_cols]
            quant_df, meta_df = df_copy[quant_cols].T, df_copy[meta_cols].T
        
        else:
            
            quant_cols = condition_df['Sample'].unique().tolist()
            meta_cols = [col for col in df_copy.columns if col not in quant_cols]
            quant_df, meta_df = df_copy[quant_cols].T, df_copy[meta_cols].T
        
        tmp_dict = dict(zip(condition_df['Sample'], condition_df['Condition']))
        quant_df.columns = meta_df.loc["PTM_Collapse_key"]
        quant_df['group'] = quant_df.index.map(tmp_dict)
        quant_df['sample'] = (quant_df['group'] + '_' + (quant_df.groupby('group').cumcount() + 1).astype(str))
        quant_df['subject'] = quant_df['sample']
        
        self.formatted_data = quant_df
        self.condition_df = condition_df
        
        return quant_df
    

In [7]:
def run_kinase_prediction (dataset, kinase_df, group1, group2, kinase_class = "ser_thr"):
    import kinase_library as kl
    ###### Preparing kinase format
    kinase_df_copy = kinase_df.copy()
    def replace_between_asterics(match):
        return match.group(1).upper()
    tmp1 = []
    for el in kinase_df_copy['kinase_sequence'].tolist():
        tmp = re.sub(r'\*([^*]+)\*', replace_between_asterics, el)
        tmp1.append(tmp.replace('_', '').upper())
        
    kinase_df_copy['kinase_sequence'] = tmp1
    kinase_df_copy['PTM_Collapse_key'] = kinase_df_copy['PTM_Collapse_key'].apply(lambda x: x.split('~')[1]).apply(lambda x: x.split('_')[0]) + '_' + 'p' + kinase_df_copy['PTM_Collapse_key'].apply(lambda x: x.split('~')[1]).apply(lambda x: x.split('_')[1])
    ###### 
    dataset_copy = dataset.copy()
    dataset_copy = dataset_copy[(dataset_copy['group'] == group1) | (dataset_copy['group'] == group2)]
    dataset_copy = dataset_copy.loc[:, dataset_copy.isna().sum() <= 24]
    dataset_copy = ac.imputation_normal_distribution(dataset_copy).reset_index()
    ttest_result = ac.run_ttest(dataset_copy, group1, group2)
    ttest_result['identifier'] = ttest_result['identifier'].apply(lambda x: x.split('~')[1]).apply(lambda x: x.split('_')[0]) + '_' + 'p' + ttest_result['identifier'].apply(lambda x: x.split('~')[1]).apply(lambda x: x.split('_')[1])
    ttest_result = ttest_result.sort_values('identifier')
    ttest_result.columns = ['PTM_Collapse_key', 'T-statistics', 'pvalue', 'mean_group1', 'mean_group2',
        'std(group1)', 'std(group2)', 'log2FC', 'test', 'correction', 'padj',
       'rejected', 'group1', 'group2', 'FC', '-log10 pvalue', 'Method']
    kinases = pd.DataFrame()
    kinases = kinase_df_copy.merge(ttest_result, on = 'PTM_Collapse_key')
    kinases = kinases[['PTM_Collapse_key','kinase_sequence', 'log2FC', 'T-statistics', 'padj']]
    kinases.columns = ['Phosphosites', 'Sequence', 'logFC', 't', 'adj.P.Val']
    test = kl.DiffPhosData(kinases, lfc_col='logFC', seq_col='Sequence', pval_col='adj.P.Val', pval_thresh=0.05)
    kin_type = kinase_class
    method = 'percentile_rank'
    thresh = 15
    test1 = test.kinase_enrichment(kin_type=kin_type, kl_method=method, kl_thresh=thresh)
    fin_df = test1.combined_enrichment_results
    return fin_df, test1

def z_normalize_data(df):
    scaler = StandardScaler()
    df_norm = pd.DataFrame(
        scaler.fit_transform(df.T).T,
        index=df.index,
        columns=df.columns
    )
    return df_norm

def perform_hierarchical_clustering(df_norm, method='ward', metric='euclidean'):

    if metric == 'correlation':
        
        sample_distances = pdist(df_norm.T, metric='correlation')
        linkage_samples = linkage(sample_distances, method='average')
    else:
        linkage_samples = linkage(df_norm.T, method=method, metric=metric)

    if metric == 'correlation':
        feature_distances = pdist(df_norm, metric='correlation')
        linkage_features = linkage(feature_distances, method='average')
    else:
        linkage_features = linkage(df_norm, method=method, metric=metric)
    
    return linkage_samples, linkage_features

def plot_clustermap(df_norm, figsize=(12, 10), cmap='rocket', 
                   method='ward', metric='euclidean', n_clusters=6, save_path=None):

    linkage_samples, linkage_features = perform_hierarchical_clustering(
        df_norm, method=method, metric=metric
    )

    sample_clusters = fcluster(linkage_samples, n_clusters, criterion='maxclust')
    feature_clusters = fcluster(linkage_features, n_clusters, criterion='maxclust')

    cluster_colors = plt.cm.Set3(np.linspace(0, 1, n_clusters))

    sample_cluster_colors = []
    for cluster in sample_clusters:
        sample_cluster_colors.append(cluster_colors[cluster - 1])

    feature_cluster_colors = []
    for cluster in feature_clusters:
        feature_cluster_colors.append(cluster_colors[cluster - 1])


    feature_cluster_palette = {i+1: cluster_colors[i] for i in range(n_clusters)}


    feature_color_array = [feature_cluster_palette[cluster] for cluster in feature_clusters]

    g = sns.clustermap(
        df_norm,
        method=method,
        metric=metric,
        cmap=cmap,
        center=0,
        figsize=figsize,
        cbar_kws={'label': 'Z-score'},
        xticklabels=True,
        yticklabels=False if df_norm.shape[0] > 50 else True,
        dendrogram_ratio=0.15,
        colors_ratio=0.03,
        row_colors=feature_color_array
    )
    
    if df_norm.shape[0] <= 100:
        for i, cluster in enumerate(np.unique(feature_clusters)):
            cluster_positions = np.where(feature_clusters == cluster)[0]
            if len(cluster_positions) > 0:
                center_pos = np.mean(cluster_positions)
                g.ax_row_colors.text(
                    0.5, center_pos, f'C{cluster}', 
                    ha='center', va='center', fontsize=8, fontweight='bold'
                )

    plt.setp(g.ax_heatmap.get_xticklabels(), rotation=45, ha='right')


    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=cluster_colors[i], 
                                   label=f'Cluster {i+1}') for i in range(n_clusters)]
    

    g.fig.legend(handles=legend_elements, title='Clusters', 
                bbox_to_anchor=(1.02, 0.8), loc='upper left')
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

    cluster_info = {
        'sample_clusters': pd.DataFrame({'Sample': df_norm.columns, 'Cluster': sample_clusters}),
        'feature_clusters': pd.DataFrame({'Feature': df_norm.index, 'Cluster': feature_clusters}),
        'clustermap': g
    }
    
    return cluster_info

def extract_clusters(linkage_matrix, labels, n_clusters=6, threshold=None):
    
    if threshold:
        clusters = fcluster(linkage_matrix, threshold, criterion='distance')
    else:
        clusters = fcluster(linkage_matrix, n_clusters, criterion='maxclust')
    
    cluster_df = pd.DataFrame({
        'Item': labels,
        'Cluster': clusters
    })
    
    return cluster_df

def prepare_gene_lists_from_clusters(feature_clusters, gene_column):
    
    if gene_column !=None and gene_column not in feature_clusters.columns:

        feature_clusters['gene_symbol'] = feature_clusters[gene_column].apply(lambda x: x.split('~')[1]).apply(lambda x: x.split('_')[0])
        for i in range(min(5, len(feature_clusters))):
            original = feature_clusters.iloc[i][gene_column]
            extracted = feature_clusters.iloc[i]['gene_symbol']
            print(f"  {original} -> {extracted}")

        failed_extractions = feature_clusters['gene_symbol'].isna().sum()
        if failed_extractions > 0:
            print(f"\nWarning: {failed_extractions} features could not be parsed for gene symbols")
            print("Examples of failed extractions:")
            failed_examples = feature_clusters[feature_clusters['gene_symbol'].isna()][gene_column].head(3).tolist()
            for example in failed_examples:
                print(f"  {example}")
            

            mask = feature_clusters['gene_symbol'].isna()

            alternative_pattern = feature_clusters.loc[mask, gene_column].str.extract(r'([A-Z][A-Z0-9]+)')
            feature_clusters.loc[mask, 'gene_symbol'] = alternative_pattern[0]
            
            remaining_failed = feature_clusters['gene_symbol'].isna().sum()
            print(f"After alternative extraction: {remaining_failed} still failed")
    
    cluster_gene_dict = {}
    
    for cluster_id in sorted(feature_clusters['Cluster'].unique()):
        genes_in_cluster = feature_clusters[
            feature_clusters['Cluster'] == cluster_id
        ][gene_column].tolist()

        genes_in_cluster = [g for g in genes_in_cluster if pd.notna(g)]
        genes_in_cluster = list(set(genes_in_cluster)) 
        
        cluster_gene_dict[f'Cluster_{cluster_id}'] = genes_in_cluster
        print(f"Cluster {cluster_id}: {len(genes_in_cluster)} unique genes")
    
    return cluster_gene_dict

def run_go_enrichment_per_cluster(cluster_gene_dict, organism='human', 
                                 gene_sets=['GO_Biological_Process_2023',
                                          'GO_Molecular_Function_2023',
                                          'GO_Cellular_Component_2023'],
                                 cutoff=0.05, top_terms=20):
    """
    Run GO enrichment analysis for each cluster
    
    Parameters:
    cluster_gene_dict: dictionary with cluster names as keys and gene lists as values
    organism: organism name ('human', 'mouse', etc.) - lowercase
    gene_sets: list of gene set databases to use
    cutoff: adjusted p-value cutoff
    top_terms: number of top terms to keep per cluster
    
    Returns:
    enrichment_results: dictionary with enrichment results for each cluster
    """
    
    enrichment_results = {}
    
    for cluster_name, gene_list in cluster_gene_dict.items():
        print(f"\nRunning enrichment for {cluster_name} ({len(gene_list)} genes)...")
        
        if len(gene_list) < 3:
            print(f"Skipping {cluster_name}: too few genes ({len(gene_list)})")
            continue
        
        try:
            # Run enrichment analysis - removed unsupported parameters
            enr = gp.enrichr(
                gene_list=gene_list,
                gene_sets=gene_sets,
                organism=organism,  # 'human', 'mouse', 'yeast', etc.
                cutoff=cutoff  # adjusted p-value cutoff
            )
            
            # Get results and filter top terms
            results_df = enr.results
            
            if not results_df.empty:
                # Sort by adjusted p-value and keep top terms
                results_df = results_df.sort_values('Adjusted P-value').head(top_terms)
                enrichment_results[cluster_name] = {
                    'results': results_df,
                    'enrichr_object': enr
                }
                print(f"Found {len(results_df)} significant terms for {cluster_name}")
            else:
                print(f"No significant terms found for {cluster_name}")
                
        except Exception as e:
            print(f"Error running enrichment for {cluster_name}: {str(e)}")
            continue
    
    return enrichment_results

def run_gsea_analysis(expression_data, gene_ranking_method='signal_to_noise', 
                     gene_sets=['GO_Biological_Process_2023'], 
                     classes=None, permutation_num=1000):
    """
    Run Gene Set Enrichment Analysis (GSEA) on expression data
    
    Parameters:
    expression_data: DataFrame with genes as rows, samples as columns
    gene_ranking_method: method to rank genes ('signal_to_noise', 'log2_ratio_of_classes', etc.)
    gene_sets: gene set databases to use
    classes: sample class labels for comparison (e.g., ['control', 'treatment', ...])
    permutation_num: number of permutations
    
    Returns:
    gsea_results: GSEA results object
    """
    
    if classes is None:
        print("Please provide class labels for GSEA analysis")
        return None
    
    print(f"Running GSEA analysis on {expression_data.shape[0]} genes...")
    
    try:
        # Run GSEA
        gsea_results = gp.gsea(
            data=expression_data,
            gene_sets=gene_sets,
            cls=classes,
            method=gene_ranking_method,
            permutation_num=permutation_num,
            outdir=None,  # Don't save to disk
            seed=42,
            verbose=True
        )
        
        print(f"GSEA completed. Found {len(gsea_results.res2d)} gene sets.")
        return gsea_results
        
    except Exception as e:
        print(f"Error running GSEA: {str(e)}")
        return None

def plot_enrichment_results(enrichment_results, top_n=10, figsize=(12, 8), save_path=None):
    """
    Create visualization of GO enrichment results
    
    Parameters:
    enrichment_results: dictionary with enrichment results from run_go_enrichment_per_cluster
    top_n: number of top terms to show per cluster
    figsize: figure size
    save_path: path to save the figure
    """
    
    # Prepare data for plotting
    plot_data = []
    
    for cluster_name, results in enrichment_results.items():
        df = results['results'].head(top_n).copy()
        df['Cluster'] = cluster_name
        df['-log10(adj_pval)'] = -np.log10(df['Adjusted P-value'])
        plot_data.append(df)
    
    if not plot_data:
        print("No enrichment results to plot")
        return
    
    combined_df = pd.concat(plot_data, ignore_index=True)
    
    # Create the plot
    plt.figure(figsize=figsize)
    
    # Dot plot style visualization
    sns.scatterplot(
        data=combined_df,
        x='Cluster',
        y='Term',
        size='-log10(adj_pval)',
        hue='Gene_set',
        sizes=(50, 400),
        alpha=0.7
    )
    
    plt.title('GO Term Enrichment Across Clusters', fontsize=16, pad=20)
    plt.xlabel('Cluster', fontsize=12)
    plt.ylabel('GO Terms', fontsize=12)
    plt.xticks(rotation=45)
    
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

def create_enrichment_heatmap(enrichment_results, top_n=15, figsize=(14, 10), save_path=None):
    """
    Create a heatmap showing enrichment across clusters
    
    Parameters:
    enrichment_results: enrichment results dictionary
    top_n: number of top terms to include
    figsize: figure size
    save_path: path to save figure
    """
    
    # Get all unique terms across clusters
    all_terms = set()
    cluster_term_pvals = {}
    
    for cluster_name, results in enrichment_results.items():
        df = results['results'].head(top_n)
        terms = df['Term'].tolist()
        pvals = df['Adjusted P-value'].tolist()
        
        all_terms.update(terms)
        cluster_term_pvals[cluster_name] = dict(zip(terms, pvals))
    
    # Create matrix for heatmap
    all_terms = list(all_terms)
    clusters = list(enrichment_results.keys())
    
    # Initialize matrix with NaN (will show as white/no enrichment)
    matrix = np.full((len(all_terms), len(clusters)), np.nan)
    
    for j, cluster in enumerate(clusters):
        for i, term in enumerate(all_terms):
            if term in cluster_term_pvals[cluster]:
                # Use -log10(p-value) for intensity
                pval = cluster_term_pvals[cluster][term]
                matrix[i, j] = -np.log10(pval) if pval > 0 else 10
    
    # Create heatmap
    plt.figure(figsize=figsize)
    
    # Mask NaN values to show them as white
    mask = np.isnan(matrix)
    
    sns.heatmap(
        matrix,
        xticklabels=clusters,
        yticklabels=all_terms,
        cmap='Reds',
        mask=mask,
        cbar_kws={'label': '-log10(Adjusted P-value)'},
        square=False
    )
    
    plt.title('GO Term Enrichment Heatmap Across Clusters', fontsize=16, pad=20)
    plt.xlabel('Clusters', fontsize=12)
    plt.ylabel('GO Terms', fontsize=12)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0, fontsize=8)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()
    
def normalize_phospho_median(phospho_df, protein_df, return_non_matched=False):
    """
    Normalize phosphoproteomics data using condition-level protein-specific normalization.

    This function normalizes each phosphosite by subtracting the median abundance of its
    corresponding parent protein within each condition. Samples with the same name are
    treated as replicates of the same condition. This approach preserves condition-specific
    protein expression changes while providing phosphorylation stoichiometry information.

    Parameters
    ----------
    phospho_df : pandas.DataFrame
        Phosphoproteomics dataframe with samples as index and phosphosites as columns.
        Column names should contain protein identifiers (e.g., 'P12345~PROTEIN_S123').
        Values should be log-transformed intensities. Duplicate sample names indicate
        replicates of the same condition.
    protein_df : pandas.DataFrame
        Proteomics dataframe with samples as index and proteins as columns.
        Column names should contain protein identifiers matching phospho data.
        Values should be log-transformed intensities. Duplicate sample names indicate
        replicates of the same condition.
    return_non_matched : bool, optional
        If True, returns all phosphosites including those without protein matches (unnormalized).
        If False (default), returns only successfully normalized phosphosites.

    Returns
    -------
    dict
        Dictionary containing:
        - 'normalized_phospho': DataFrame with protein-normalized phospho data
        - 'condition_protein_medians': DataFrame with median protein values per condition
        - 'phospho_to_protein_mapping': Dict mapping phosphosites to protein IDs
        - 'normalization_success_rate': Float indicating fraction successfully normalized
        - 'common_conditions': List of conditions present in both datasets

    Notes
    -----
    - Protein IDs are extracted from phosphosite names using '~' or '_' delimiters
    - For each condition, calculates median protein abundance across replicates
    - Each phosphosite normalized by its parent protein's median in that condition
    - By default, only returns phosphosites with successful protein matches
    - Set return_non_matched=True to include unmatched phosphosites (unnormalized)
    - Preserves biological protein expression differences between conditions

    Raises
    ------
    ValueError
        If no common conditions are found between the two datasets

    Examples
    --------
    >>> results = normalize_phospho_median(phospho_data, protein_data)
    >>> normalized_data = results['normalized_phospho']
    >>> success_rate = results['normalization_success_rate']
    >>> print(f"Successfully normalized {success_rate:.1%} of phosphosites")
    """
    # Find common conditions
    phospho_conditions = set(phospho_df.index)
    protein_conditions = set(protein_df.index)
    common_conditions = phospho_conditions & protein_conditions

    if len(common_conditions) == 0:
        raise ValueError("No common conditions found between phospho and protein data!")

    print(f"Found {len(common_conditions)} common conditions: {sorted(common_conditions)}")

    # Filter to common conditions
    phospho_matched = phospho_df.loc[phospho_df.index.isin(common_conditions)]
    protein_matched = protein_df.loc[protein_df.index.isin(common_conditions)]

    print(f"Phospho samples: {len(phospho_matched)}")
    print(f"Protein samples: {len(protein_matched)}")

    # Extract protein IDs from phosphosite names
    def extract_protein_id(phosphosite_name):
        """Extract protein ID from phosphosite name (format: 'A0A087WUL8~NBPF19_S364_M1')"""
        return (
            phosphosite_name.split("~")[0]
            if "~" in phosphosite_name
            else phosphosite_name.split("_")[0]
        )

    # Create phosphosite to protein mapping
    phospho_to_protein = {}
    for phosphosite in phospho_matched.columns:
        protein_id = extract_protein_id(phosphosite)
        phospho_to_protein[phosphosite] = protein_id

    print(f"Mapped {len(phospho_to_protein)} phosphosites to proteins")

    # Calculate condition-level protein medians
    # Group protein data by condition and calculate median for each protein
    condition_protein_medians = protein_matched.groupby(protein_matched.index).median()

    print(f"Calculated protein medians for {len(condition_protein_medians)} conditions")
    print(f"Protein medians shape: {condition_protein_medians.shape}")

    # Normalize phospho data
    normalized_phospho = phospho_matched.copy()
    successfully_normalized_phosphosites = set()
    normalization_stats = {
        "total_values": 0,
        "normalized": 0,
        "protein_not_found": 0,
        "missing_protein_values": 0,
    }

    for condition in common_conditions:
        condition_mask = phospho_matched.index == condition
        condition_phospho = phospho_matched.loc[condition_mask]

        for phosphosite in phospho_matched.columns:
            protein_id = phospho_to_protein[phosphosite]
            normalization_stats["total_values"] += sum(condition_mask)

            # Find matching protein column
            matching_proteins = [
                col for col in condition_protein_medians.columns if protein_id in col.split(";")
            ]

            if matching_proteins:
                protein_col = matching_proteins[0]
                condition_protein_median = condition_protein_medians.loc[condition, protein_col]

                if not pd.isna(condition_protein_median):
                    # Normalize all replicates of this condition for this phosphosite
                    normalized_phospho.loc[condition_mask, phosphosite] = (
                        condition_phospho[phosphosite] - condition_protein_median
                    )
                    normalization_stats["normalized"] += sum(condition_mask)
                    successfully_normalized_phosphosites.add(phosphosite)
                else:
                    normalization_stats["missing_protein_values"] += sum(condition_mask)
            else:
                normalization_stats["protein_not_found"] += sum(condition_mask)

    # Filter to only successfully normalized phosphosites (unless return_non_matched=True)
    if not return_non_matched:
        print("Filtering to only successfully normalized phosphosites...")
        print(f"Original phosphosites: {len(phospho_matched.columns)}")
        print(f"Successfully normalized: {len(successfully_normalized_phosphosites)}")
        print(
            f"Removed: {len(phospho_matched.columns) - len(successfully_normalized_phosphosites)}"
        )

        normalized_phospho = normalized_phospho[list(successfully_normalized_phosphosites)]

        # Also update the original phospho for consistency
        phospho_matched = phospho_matched[list(successfully_normalized_phosphosites)]

    # Calculate success rate
    success_rate = (
        normalization_stats["normalized"] / normalization_stats["total_values"]
        if normalization_stats["total_values"] > 0
        else 0
    )

    print("\nNormalization statistics:")
    print(f"  Total phosphosite-sample combinations: {normalization_stats['total_values']:,}")
    print(f"  Successfully normalized: {normalization_stats['normalized']:,} ({success_rate:.1%})")
    print(f"  Protein not found: {normalization_stats['protein_not_found']:,}")
    print(f"  Missing protein values: {normalization_stats['missing_protein_values']:,}")

    return {
        "normalized_phospho": normalized_phospho,
        "condition_protein_medians": condition_protein_medians,
        "phospho_to_protein_mapping": phospho_to_protein,
        "normalization_success_rate": success_rate,
        "common_conditions": list(common_conditions),
        "original_phospho": phospho_matched,
        "original_protein": protein_matched,
    }

def replace_between_asterics(match):
    return match.group(1).upper()

def get_protein_input(y):
    return math.exp((y+17794.1)/7000.7)

# Data import 

In [6]:
shapes_100 = pd.read_csv(r'W:\User\Denys\SN_diffs\DVPP_100shapes_out_Report_phosphopeptides (Normal).tsv', sep = '\t')
shapes_200 = pd.read_csv(r'W:\User\Denys\SN_diffs\DVPP_200shapes_out_Report_phosphopeptides (Normal).tsv', sep = '\t')
shapes_300 = pd.read_csv(r'W:\User\Denys\SN_diffs\DVPP_300shapes_out_Report_phosphopeptides (Normal).tsv', sep = '\t')
shapes_500 = pd.read_csv(r'W:\User\Denys\SN_diffs\DVPP_500shapes_out_Report_phosphopeptides (Normal).tsv', sep = '\t')
shapes_1000 = pd.read_csv(r'W:\User\Denys\SN_diffs\DVPP_1000shapes_out_Report_phosphopeptides (Normal).tsv', sep = '\t')
shapes_1500 = pd.read_csv(r'W:\User\Denys\SN_diffs\DVPP_1500shapes_out_Report_phosphopeptides (Normal).tsv', sep = '\t')

In [9]:
shapes_list = [shapes_100, shapes_200, shapes_300, shapes_500, shapes_1000, shapes_1500]

In [ ]:
df_pDVP = pd.read_csv(r'W:\User\Denys\SN_diffs\pDVP_neurons_all.tsv', sep = '\t')
df_DVP = pd.read_csv(r'W:\User\Denys\SN_diffs\DVP_neurons_all.tsv', sep = '\t')
df_DVP = df_DVP.set_index('PG.ProteinGroups')

In [ ]:
DVP_condition = pd.read_csv(r'W:\User\Denys\SN_diffs\DVP_neurons_conditionSetup.tsv',sep = '\t')

In [ ]:
df = pd.read_csv(r'W:\User\Denys\SN_diffs\pDVP_neurons_all_aq.tsv',sep = '\t')

# PeptideCollapse on data

In [ ]:
shapes_list_collapsed = []
pc = PeptideCollapse()
for i, l in enumerate(shapes_list):
    shapes_list_collapsed.append(pc.process_complete_pipeline(l, cutoff=0, fasta_path=r'D:\Projects\Spectral libraries\mouse.fasta'))

In [ ]:
builder = PhosphoAnalysis()
df_pDVP_collapsed = builder.peptideCollapse(df_pDVP, cutoff=0, fasta_path=r'D:\Projects\Spectral libraries\mouse.fasta')

# Figure 5b

In [11]:
figure = get_cumulative_barplot(shapes_list_collapsed, 4, point_size=9, point_color='black')
#figure.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5b.pdf', width = 600, height = 600)

# Calculate the theoretical amount of protein in shapes (Suppl. Figure 4a with equation from Suppl. Figure 3a)

In [158]:
est_input = []

for l in shapes_list_collapsed:
    est_input.append(np.round(get_protein_input(len(l)), 0))

In [159]:
est_input

[14.0, 18.0, 21.0, 33.0, 55.0, 95.0]

In [160]:
labels = [100, 200, 300, 500, 1000, 1500]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = labels, y = est_input, marker_color = '#ef755d'))
fig.update_layout(width = 600, height = 600, template = 'plotly_white')
fig.update_traces(marker = dict(size = 14))
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\suppl_figure4b.pdf', height = 600, width = 600)

# Supplementary Figure 4b

In [ ]:
cv_list = []
for df in shapes_list_collapsed:
    linear_values = np.power(2, df.iloc[:,:3])

    n_valid = linear_values.notna().sum(axis=1)
    
    cv = linear_values.std(axis=1) / linear_values.mean(axis=1)
    cv = cv.replace(0, np.nan)

    cv[n_valid < 3] = np.nan
    
    cv_list.append(cv)

In [163]:
cv_list_flattened = [item for sublist in cv_list for item in sublist]

In [164]:
np.nanmedian(cv_list_flattened)

0.18160456299884248

In [166]:
color_palette = ['#EEA69B', '#E78373', '#E1604C', '#DB452E', '#B3321E', '#8C2718', '#641C11']

In [ ]:
fig = go.Figure()
for i, el in enumerate(cv_list):
    fig.add_trace(go.Box(y = el, marker_color = color_palette[i]))
fig.update_layout(width = 900, height = 600, showlegend = False, template = 'plotly_white')
fig.add_hline(y = np.nanmedian(cv_list_flattened), line = {'dash':'dash', 'width': 2, 'color':'black'})
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\suppl_figure4c.pdf', height = 600, width = 900)

# Figure 5c

In [14]:
a = calculate_interdilution_correlation(shapes_list_collapsed, shapes_list_collapsed[-1], est_input, nreps = 4)

In [18]:
np.nanmedian(a)

0.8689795745327651

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=a,
    nbinsx=len(a)//7,  # Dynamic bin count
    name='PG Number',
    marker=dict(
        color='black',
        line=dict(color='black', width=0.3)
    ),
    opacity=0.8
))
fig.add_vline(x = np.nanmedian(a), line={'dash': 'dash', 'width': 3, 'color': 'darkred'})
fig.update_layout(
    width=600, 
    height=600, 
    template='plotly_white',
    xaxis_title='R squared',
    yaxis_title='Count',
    font=dict(size=12),
    showlegend=False,
    bargap=0.05
)
#fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5c.pdf', height = 600, width = 600)

# Figure 5d

In [14]:
def find_common_phosphosites(list_of_dfs, key_col='PTM_Collapse_key'):

    phosphosite_sets = [set(df[key_col].dropna().unique()) for df in list_of_dfs]

    common_phosphosites = set.intersection(*phosphosite_sets)
    
    return common_phosphosites

common_sites = find_common_phosphosites(shapes_list_collapsed)

In [15]:
sites_to_include = ['P20357~Map2_S1783_M1', 'P10637~Mapt_S494_M1', 'P28652~Camk2b_T287_M1',  'P63318~Prkcg_T514_M1']

In [16]:
vals = []

for site in sites_to_include:
    tmp1 = []
    for df in shapes_list_collapsed:
        tmp2 = []
        tmp3 = list(df[df['PTM_Collapse_key'] == site].iloc[:,:4].values[0])
        tmp2.append(tmp3)
        tmp1.append(tmp3)
    vals.append(tmp1)

In [87]:
color_palette = ["#ef745c","#d06257","#b15052","#923e4d","#722b47","#531942","#34073d"]

In [90]:
fig = go.Figure()
for i, el in enumerate(vals[0]):
    fig.add_trace(go.Box(y = el, boxpoints='all',fillcolor = color_palette[i], pointpos=0, showlegend=False, marker=dict(color='black',size=8, opacity=1)))
fig.update_layout(width = 600, height = 400, template = 'none')
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5d1.pdf', height = 400, width = 600)

In [92]:
fig = go.Figure()
for i, el in enumerate(vals[1]):
    fig.add_trace(go.Box(y = el, boxpoints='all',fillcolor = color_palette[i], pointpos=0, showlegend=False, marker=dict(color='black',size=8, opacity=1)))
fig.update_layout(width = 600, height = 400, template = 'none')
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5d2.pdf', height = 400, width = 600)

In [96]:
fig = go.Figure()
for i, el in enumerate(vals[2]):
    fig.add_trace(go.Box(y = el, boxpoints='all',fillcolor = color_palette[i], pointpos=0, showlegend=False, marker=dict(color='black',size=8, opacity=1)))
fig.update_layout(width = 600, height = 400, template = 'none')
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5d3.pdf', height = 400, width = 600)

In [95]:
fig = go.Figure()
for i, el in enumerate(vals[3]):
    fig.add_trace(go.Box(y = el, boxpoints='all',fillcolor = color_palette[i], pointpos=0, showlegend=False, marker=dict(color='black',size=8, opacity=1)))
fig.update_layout(width = 600, height = 400, template = 'none')
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5d4.pdf', height = 400, width = 600)

# Figure 5e

In [ ]:

DVP_condition['Condition'] = DVP_condition['Condition'].apply(lambda x: x[:-2])
DVP_condition['Run Label'] = DVP_condition['Run Label'].apply(lambda x: x.split('.')[0])

In [268]:
condition_proteome = pd.DataFrame({'Sample':DVP_condition['Run Label'], 'Condition':DVP_condition['Condition']})

In [269]:
df_DVP.columns = condition_proteome['Condition']

In [270]:
df_DVP = np.log2(df_DVP)

In [248]:
meta = df_pDVP_collapsed.iloc[:,-6:]

In [249]:
nums = []
labels = []
for el in df_pDVP_collapsed.iloc[:,:-6].columns:
    nums.append(len(df_pDVP_collapsed[el].dropna()))
    labels.append(el)
d = dict(zip(labels, nums))

In [250]:
threshold = np.nanmedian(list(d.values())) - stats.iqr(list(d.values()))*1.5

In [251]:
columns_to_keep = [col for col, count in d.items() if count >= threshold] + list(meta.columns)
columns_to_remove = [col for col, count in d.items() if count <= threshold]

In [ ]:
df_pDVP_collapsed = df_pDVP_collapsed[columns_to_keep] #removing outliers with very low coverage

In [252]:
pDVP_condition = pd.read_csv(r'W:\User\Denys\SN_diffs\pDVP_neurons_conditionSetup.tsv',sep = '\t')

In [253]:
pDVP_condition['Condition'] = pDVP_condition['Condition'].apply(lambda x: x[:-1])
pDVP_condition['Run Label'] = pDVP_condition['Run Label'].apply(lambda x: x.split('.')[0])

In [254]:
condition_phospho = pd.DataFrame({'Sample':pDVP_condition['Run Label'], 'Condition':pDVP_condition['Condition']})

In [256]:
condition_phospho = condition_phospho[condition_phospho['Sample'].isin(columns_to_keep)]

In [260]:
df_pDVP_collapsed_cond = builder.assign_condition_setup(condition_phospho)

In [263]:
df_pDVP_collapsed_filt = df_pDVP_collapsed_cond.loc[:, (1 - (df_pDVP_collapsed_cond.isna().sum() / len(df_pDVP_collapsed_cond))) >=0.7]

In [265]:
df_pDVP_collapsed_imp = ac.imputation_normal_distribution(df_pDVP_collapsed_filt).reset_index()

In [ ]:
df_pDVP_collapsed_imp1 = df_pDVP_collapsed_imp.drop(['sample','subject'], axis = 1).set_index('group')
df_phospho_normalized = normalize_phospho_median(df_pDVP_collapsed_imp1, df_DVP.T)

In [272]:
df_phospho_normalized = df_phospho_normalized['normalized_phospho']

In [273]:
df_phospho_normalized = df_phospho_normalized.reset_index()
df_phospho_normalized['sample'] = df_pDVP_collapsed_imp['sample']
df_phospho_normalized['subject'] = df_pDVP_collapsed_imp['subject']

In [274]:
pca = ac.run_pca(df_phospho_normalized)

In [277]:
pca[1]

{'x_title': 'PC1 (0.14)', 'y_title': 'PC2 (0.11)', 'group': 'group'}

In [282]:
color_palette = ['#2a9d8f', '#f4a261', '#264653', '#e76f51']

In [ ]:
fig = px.scatter(pca[0][0], x='x', y='y', labels="sample", color='group', hover_name='sample', color_discrete_sequence= color_palette)

fig.update_layout(
    width=600, 
    height=600, 
    template='plotly_white',
    xaxis_title=pca[1]['x_title'],
    yaxis_title=pca[1]['y_title']
)

fig.update_traces(marker=dict(
    size=21, 
    line=dict(width=1, color='black')
))

fig.show()
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5e.pdf', height = 600, width = 650)

# Figure 5f

In [286]:
anova = ac.run_anova(df_phospho_normalized)

d:\Projects\nanoPhos_env\Lib\site-packages\pingouin\pairwise.py:28: UserWarning:

pairwise_ttests is deprecated, use pairwise_tests instead.

2025-12-02 15:32:22,586 - choreographer.utils._tmpfile - INFO - TemporaryDirectory.cleanup() worked.
2025-12-02 15:32:22,587 - choreographer.utils._tmpfile - INFO - shutil.rmtree worked.
d:\Projects\nanoPhos_env\Lib\site-packages\pingouin\pairwise.py:28: UserWarning:

pairwise_ttests is deprecated, use pairwise_tests instead.

d:\Projects\nanoPhos_env\Lib\site-packages\pingouin\pairwise.py:28: UserWarning:

pairwise_ttests is deprecated, use pairwise_tests instead.

d:\Projects\nanoPhos_env\Lib\site-packages\pingouin\pairwise.py:28: UserWarning:

pairwise_ttests is deprecated, use pairwise_tests instead.

d:\Projects\nanoPhos_env\Lib\site-packages\pingouin\pairwise.py:28: UserWarning:

pairwise_ttests is deprecated, use pairwise_tests instead.

d:\Projects\nanoPhos_env\Lib\site-packages\pingouin\pairwise.py:28: UserWarning:

pairwise_ttests is de

In [308]:
ttest_sig = anova[anova['rejected'] == True]
meta = df_pDVP_collapsed_imp[['group', 'sample', 'subject']]
df_pDVP_collapsed_imp_sig = df_pDVP_collapsed_imp.groupby('group').mean(numeric_only=True)
df_pDVP_collapsed_imp_sig = df_pDVP_collapsed_imp_sig.T.reset_index()
df_pDVP_collapsed_imp_sig = df_pDVP_collapsed_imp_sig[df_pDVP_collapsed_imp_sig ['PTM_Collapse_key'].isin(ttest_sig['identifier'])]
df_pDVP_collapsed_imp_sig = df_pDVP_collapsed_imp_sig.set_index('PTM_Collapse_key')

In [309]:
df_norm = z_normalize_data(df_pDVP_collapsed_imp_sig)
linkage_samples, linkage_features = perform_hierarchical_clustering(
        df_norm, method='ward', metric='euclidean'
    )

In [ ]:
clustermap = plot_clustermap(df_norm, figsize=(3, 10), method='ward', 
                                metric='euclidean', n_clusters=4, save_path=r'D:\Projects\nanoPhos\figures_upd\figure5\figure5f.pdf')

In [363]:
feature_clusters = extract_clusters(linkage_features, df_norm.index.tolist(),  n_clusters=4)
feature_clusters['gene_symbol'] = feature_clusters['Item'].apply(lambda x: x.split('~')[1]).apply(lambda x: x.split('_')[0])
feature_clusters.columns = ['PTM_Collapse_key', 'Cluster', 'gene_symbol']

In [364]:
df_norm_clus = df_norm.reset_index()
df_norm_clus = df_norm_clus.merge(feature_clusters, on = 'PTM_Collapse_key').set_index('PTM_Collapse_key')

In [365]:
custom_order = ['act_subcortex', 'inh_subcortex', 'act_cortex', 'inh_cortex','Cluster', 'gene_symbol']
df_norm_clus = df_norm_clus.reindex(columns=custom_order)

In [366]:
df_norm_clus2 = df_norm_clus[(df_norm_clus['Cluster'] == 1)].drop(['Cluster', 'gene_symbol'], axis = 1)
df_norm_clus5 = df_norm_clus[(df_norm_clus['Cluster'] == 3)].drop(['Cluster', 'gene_symbol'], axis = 1)

In [367]:
length_clus2 = []
for col in df_norm_clus2.columns:
    length_clus2.append(np.mean(df_norm_clus2[col].tolist()))

length_clus5 = []
for col in df_norm_clus5.columns:
    length_clus5.append(np.mean(df_norm_clus5[col].tolist()))

In [ ]:
fig = go.Figure()
for index, row in df_norm_clus2.iterrows():
    fig.add_trace(go.Scatter(x = df_norm_clus2.columns, y = row, mode = 'lines', marker_color = 'lightgrey', opacity=0.5, showlegend=False, line=dict(color='lightgrey', width=0.3)))
fig.add_trace(go.Scatter(x = df_norm_clus2.columns, y = length_clus2, mode = 'lines', marker_color = 'red', opacity=1, showlegend=False, line=dict(color='red', width=0.3)))
fig.update_layout(width= 600, height = 450, template = 'plotly_white')
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5fp1.pdf', height = 450, width = 600)

In [ ]:
fig = go.Figure()
for index, row in df_norm_clus5.iterrows():
    fig.add_trace(go.Scatter(x = df_norm_clus5.columns, y = row, mode = 'lines', marker_color = 'lightgrey', opacity=0.5, showlegend=False, line=dict(color='lightgrey', width=0.3)))
fig.add_trace(go.Scatter(x = df_norm_clus5.columns, y = length_clus5, mode = 'lines', marker_color = 'red', opacity=1, showlegend=False, line=dict(color='red', width=0.3)))
fig.update_layout(width= 600, height = 450, template = 'plotly_white')
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5fp2.pdf', height = 450, width = 600)

In [370]:
lib = gp.get_library_name(organism='Mouse')

In [371]:
lib_sub = ['GO_Molecular_Function_2025','GO_Cellular_Component_2025', 'GO_Biological_Process_2025', 'KEGG_2019_Mouse']

In [372]:
tmp1 = []
tmp2 = []

for c in feature_clusters['Cluster'].unique().tolist():
    tmp_df = feature_clusters[feature_clusters['Cluster'] == c]
    tmp1.append('Cluster'+'_'+str(c))
    tmp2.append(tmp_df['gene_symbol'].unique().tolist())


fin_dict = dict(zip(tmp1, tmp2))

In [ ]:
cluster_gene_dict = prepare_gene_lists_from_clusters(feature_clusters, 'gene_symbol')

In [ ]:
enrichment_results = run_go_enrichment_per_cluster(
        fin_dict,
        organism='Mouse',
        gene_sets=['WikiPathways_2024_Mouse', 'GO_Molecular_Function_2025','GO_Cellular_Component_2025', 'GO_Biological_Process_2025', 'KEGG_2019_Mouse', 'SynGO_2024'],
        top_terms=200, cutoff=0.05
    )

In [375]:
clus_2_go = enrichment_results['Cluster_1']['results']
clus_2_go = clus_2_go[clus_2_go['Adjusted P-value']<0.05]
####
clus_5_go = enrichment_results['Cluster_3']['results']
clus_5_go = clus_5_go[clus_5_go['Adjusted P-value']<0.05]

In [376]:
list_cluster2 = ['Synaptic Vesicle Cycle (GO:0099504) BP', 'Structural Constituent Of Active Zone (GO:0098882) BPp', 'Synaptic Vesicle Clustering (GO:0097091) BP','Inositol-1,4,5-Trisphosphate 5-Phosphatase Activity (GO:0052658)','Perineuronal Net (GO:0072534)', 'Neuron Development (GO:0048666)']
list_cluster5 = ['Calcium/Calmodulin-Dependent Protein Kinase Activity (GO:0004683)', 'Protein Sumoylation (GO:0016925)', 'Positive Regulation of Translation (GO:0045727)', 'Chromatin Organization (GO:0006325)', 'Neuron Projection (GO:0043005)']

In [377]:
cluster2_df = clus_2_go[clus_2_go['Term'].isin(list_cluster2)].sort_values('Combined Score', ascending = True)
cluster5_df = clus_5_go[clus_5_go['Term'].isin(list_cluster5)].sort_values('Combined Score', ascending = True)

In [ ]:
fig = px.bar(cluster2_df, y = 'Term', x = np.log2(cluster2_df['Combined Score']), color = 'Adjusted P-value', orientation='h', color_continuous_scale=[
                 [0, '#5a4a6f'],      # Very dark violet (low p-value = significant)
                 [0.25, '#9970ab'],   # Dark violet
                 [0.5, '#c994c7'],    # Medium violet
                 [0.75, '#d4b9da'],   # Light violet
                 [1, '#e0d0e8']       # More visible light violet (high p-value)
             ])
fig.update_layout(width = 600, height = 600, template = 'none')
#fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5fp3.pdf', height = 600, width = 600)

In [ ]:
fig = px.bar(cluster5_df, y = 'Term', x = np.log2(cluster5_df['Combined Score']), color = 'Adjusted P-value', orientation='h', color_continuous_scale=[
                 [0, '#5a4a6f'],      # Very dark violet (low p-value = significant)
                 [0.25, '#9970ab'],   # Dark violet
                 [0.5, '#c994c7'],    # Medium violet
                 [0.75, '#d4b9da'],   # Light violet
                 [1, '#e0d0e8']       # More visible light violet (high p-value)
             ])
fig.update_layout(width = 600, height = 600, template = 'none')
#fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\figure5fp4.pdf', height = 600, width = 600)

# Supplementary Figure 4e 

In [33]:
samplemap_phospho = pd.DataFrame({'sample': condition_df['Sample'], 'condition': condition_df['Condition']})
samplemap_phospho.to_csv(r'D:\Projects\nanoPhos\figures_upd\figure5\aq_out\samplemap_phospho.tsv', sep= '\t', index = False)

In [408]:
df1 = df[df['R.FileName'].isin(condition_phospho['Sample'])]

In [410]:
tmp1 = df1['R.FileName'].unique().tolist()
tmp2 = []
for el in tmp1:
    value = condition_phospho[condition_phospho['Sample'].isin([el])]['Condition'].values
    if len(value) > 0:
        tmp2.append(str(value[0]))
    else:
        tmp2.append('') 

samplemap_phospho = pd.DataFrame({'sample': tmp1, 'condition': tmp2})

In [415]:
df1.to_csv(r'D:\pDVP_phospho_neurons_AQ.tsv', sep = '\t')

In [424]:
samplemap_phospho = pd.DataFrame({'sample': df1['R.Label'].unique(), 'condition': samplemap_phospho['condition']})

In [429]:
samplemap_phospho.to_csv(r'D:\Projects\nanoPhos\figures_upd\figure5\aq_out\samplemap_phospho.tsv', sep= '\t')

In [430]:
PHOSPHO_FILE = r'D:\pDVP_phospho_neurons_AQ.tsv'
SAMPLEMAP_PHOSPHO = r'D:\Projects\nanoPhos\figures_upd\figure5\aq_out\samplemap_phospho.tsv'
RESULTS_DIR_PHOSPHO = r'D:\Projects\nanoPhos\figures_upd\figure5\aq_out'
CONDPAIRS_LIST = [('act_cortex', 'inh_cortex')]

In [ ]:
aq_pipeline.run_pipeline(input_file=PHOSPHO_FILE, samplemap_file=SAMPLEMAP_PHOSPHO, results_dir=RESULTS_DIR_PHOSPHO,
                        condpairs_list=CONDPAIRS_LIST, perform_ptm_mapping=True,modification_type="[Phospho (STY)]",organism="mouse", volcano_fcthresh = 0.5) 

In [4]:
ttest_res = pd.read_csv(r'D:\Projects\nanoPhos\figures_upd\figure5\aq_out\act_cortex_VS_inh_cortex.results.tsv',sep = '\t')

In [5]:
sig = ttest_res[ttest_res['color'] == 'green']

In [ ]:
tmp = []
for index, row in ttest_res.iterrows():
    if (row['log2fc']>0.5) & (row['color'] == 'green'):
        tmp.append('up')
    elif (row['log2fc']<-0.5) & (row['color'] == 'green'):
        tmp.append('down')
    else:
        tmp.append('none')

In [ ]:
fig = px.scatter(ttest_res, x = 'log2fc', y = '-log10(fdr)', color = 'ID',color_discrete_sequence=['#d5d9d8', '#d3321d', '#5257e5'])
fig.update_layout(width = 600, height = 600,template = 'plotly_white')
fig.update_traces(marker = dict(size = 13, line = dict(width = 0.5)))
fig.write_image(r'D:\Projects\nanoPhos\figures_upd\figure5\suplp_figure4e.png', height = 600, width = 600,scale = 5)